In [1]:
import polars as pl
from db.model import Tactics, Techniques, TacTech, Datasources, TechData, DataCollection, Cyberkillchain
from miter.create_db import create_tactic_as_records, create_technique_as_records

In [8]:
tactic_path = "/home/work/data/enterprise-attack-v15.1-tactics.xlsx"
technique_path = "/home/work/data/enterprise-attack-v15.1-techniques.xlsx"

In [7]:
target_model = Techniques

In [9]:
target_model.drop_table()
target_model.create_table()

In [10]:
data = create_technique_as_records(technique_path)

/home/work/src/miter/create_db.py:67: PolarsInefficientMapWarning: 
Expr.map_elements is significantly slower than the native expressions API.
Only use if you absolutely CANNOT implement your logic otherwise.
Replace this expression...
  - pl.col("ID").map_elements(lambda x: ...)
with this one instead:
  + pl.col("ID").replace(TACTIC_ORDER_DICT)

  df = df.with_columns(pl.col("ID").apply(lambda x: TACTIC_ORDER_DICT[x]).alias("sequence")).sort("sequence")
/home/work/src/miter/create_db.py:67: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns(pl.col("ID").apply(lambda x: TACTIC_ORDER_DICT[x]).alias("sequence")).sort("sequence")


ComputeError: KeyError: 'T1548'

In [6]:
target_model.insert_many(data).execute()

0